In [1]:
# Ignoring Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas_datareader as web
import pandas as pd
from yahoo_fin import stock_info as si
import datetime as dt
# yfinance is where we will pull our data from
import yfinance as yf
%matplotlib inline
import matplotlib.pyplot as plt 
plt.style.use("seaborn")

# Normalizing the Adjusted Close column and Saving into the Database

In [2]:
# Reading the stocks_adj_close.csv
stocks = pd.read_csv("stocks_adj_close.csv", index_col = [0], parse_dates = [0])
stocks.head()

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XRAY,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-05,19.963375,5.005957,37.928116,6.556004,21.944105,8.53,19.619041,33.540310,37.700001,23.353294,...,54.343529,29.609983,45.093864,13.622756,43.354244,32.392876,18.730406,54.986431,28.620001,11.777194
2010-01-06,19.892447,4.798555,38.258835,6.451721,21.736616,8.40,19.727999,33.896866,37.619999,23.308975,...,55.218384,29.282593,44.502361,13.648920,43.728966,32.606281,18.596510,54.968670,28.400000,12.801298
2010-01-07,19.866661,4.939965,38.249397,6.439795,21.388035,8.40,19.891426,33.865170,36.889999,23.124332,...,55.055614,29.151625,45.452759,13.590055,43.591572,33.033108,18.591158,56.229664,27.690001,14.235043
2010-01-08,19.860210,4.845691,38.400570,6.482610,21.620428,8.23,19.993120,33.730465,36.689999,23.257280,...,54.994595,28.883154,45.127102,13.596595,43.416714,33.033108,18.596510,55.048595,27.600000,14.004617
2010-01-11,19.873106,4.751417,38.022617,6.425421,21.861107,8.08,20.094807,33.698780,36.209999,23.124332,...,55.543926,29.073048,44.980888,13.727399,43.903843,33.459942,18.917879,56.265182,27.690001,14.089961


In [3]:
# Getting the tickers list
tickers = list(stocks.columns)

In [4]:
# As you can see all the stocks are not starting from same y value, 
# lets normalize the data and then plot and see the price fluctuations
# we will divide every row by the first row and multiply by 100 to covert it into %
# for example the following code is printing the normalized values of first column
# Lets normalize all the columns
norm_adj_close = (stocks/(stocks.iloc[0]))*100
norm_adj_close.head()

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XRAY,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-05,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
2010-01-06,99.644707,95.856892,100.871963,98.409361,99.054466,98.475966,100.555365,101.063067,99.787794,99.810223,...,101.609861,98.894323,98.688285,100.192061,100.864325,100.658804,99.285142,99.967699,99.231303,108.695655
2010-01-07,99.515543,98.681733,100.847080,98.227448,97.465969,98.475966,101.388369,100.968565,97.851455,99.019573,...,101.310341,98.452013,100.795883,99.759957,100.547415,101.976460,99.256568,102.260981,96.750523,120.869560
2010-01-08,99.483230,96.798486,101.245657,98.880510,98.524993,96.482999,101.906713,100.566945,97.320949,99.588863,...,101.198056,97.545323,100.073707,99.807960,100.144091,101.976460,99.285142,100.113054,96.436057,118.913017
2010-01-11,99.547827,94.915259,100.249159,98.008202,99.621774,94.724504,102.425018,100.472477,96.047741,99.019573,...,102.208906,98.186639,99.749465,100.768148,101.267693,103.294139,101.000904,102.325576,96.750523,119.637675


In [ ]:
# Lets plot the normalized closed values
# norm_adj_close.plot(figsize = (15,8), fontsize = 12)
# plt.legend(loc = "upper left", fontsize = 12)
# plt.show()

In [6]:
# Merging into Database
for ticker in tickers:
    df = pd.read_csv(f"stock_data/{ticker}.csv", index_col = [0], parse_dates = [0]) 
    df['Normalized Adj Close'] = norm_adj_close[ticker]
    df.to_csv(f'stock_data/{ticker}.csv')

In [7]:
start = '2010-01-01'
end = '2022-04-08'
# Load sp500_df return data for comparing the returns of individual stocks
sp500_df = web.DataReader('^GSPC', 'yahoo', start, end)
sp500_df['Percent Change'] = sp500_df['Adj Close'].pct_change()
sp500_return = (sp500_df['Percent Change'] + 1).cumprod()[-1]

In [8]:
sp500_df.head()

,High,Low,Open,Close,Volume,Adj Close,Percent Change
Date,,,,,,,
2010-01-04,1133.869995,1116.560059,1116.560059,1132.989990,3991400000,1132.989990,NaN
2010-01-05,1136.630005,1129.660034,1132.660034,1136.520020,2491020000,1136.520020,0.003116
2010-01-06,1139.189941,1133.949951,1135.709961,1137.140015,4972660000,1137.140015,0.000546
2010-01-07,1142.459961,1131.319946,1136.270020,1141.689941,5270680000,1141.689941,0.004001
2010-01-08,1145.390015,1136.219971,1140.520020,1144.979980,4389590000,1144.979980,0.002882


In [9]:
sp500_return

3.9614469888015336

In [10]:
# Lets figure out the best performers by comparing with the spy returns
# We will assign
return_list = []
for ticker in tickers:
    df = stocks[ticker]
    df['Percent Change'] = df.pct_change()
    stock_return = ((df['Percent Change'] + 1).cumprod())[-1]
    returns_compared = round((stock_return / sp500_return),2)
    return_list.append(returns_compared)

In [11]:
best_performers = pd.DataFrame(list(zip(tickers, return_list)), columns = ['Ticker', 'Returns Compared'])

In [12]:
best_performers

,Ticker,Returns Compared
0,A,1.71
1,AAL,0.84
2,AAP,1.50
3,AAPL,6.55
4,ABC,1.90
...,...,...
438,XRAY,0.37
439,YUM,1.61
440,ZBH,0.60
441,ZBRA,3.67


In [13]:
# Showing the ranking in percentile form
best_performers['Score'] = best_performers['Returns Compared'].rank(pct = True)*100

In [14]:
best_performers.head()

,Ticker,Returns Compared,Score
0,A,1.71,61.851016
1,AAL,0.84,23.363431
2,AAP,1.50,53.047404
3,AAPL,6.55,97.291196
4,ABC,1.90,67.945824


In [15]:
# Setting the condition of picking the stocks which are in top 70% quantile 
# We can do the same thing with sharpe ratio and other factors
best_performers = best_performers[best_performers['Score'] >= (best_performers['Score'].quantile(0.70))]

In [16]:
best_performers.sort_values(by = 'Score', ascending = False)

,Ticker,Returns Compared,Score
138,DXCM,14.70,100.000000
297,NVDA,13.54,99.774266
130,DPZ,13.53,99.548533
287,NFLX,12.21,99.322799
43,AVGO,10.18,99.097065
...,...,...,...
284,NDSN,2.04,70.993228
207,INCY,2.04,70.993228
86,CINF,2.03,70.654628
12,ADSK,2.02,70.316027


In [17]:
len(best_performers)

133

In [18]:
final_df = pd.DataFrame(columns = ['Tickers', 'Latest_Price', 'Score', 'PE_Ratio','PEG_Ratio','SMA_150', 'SMA_200', '52_week_low', '52_week_high'])

In [19]:
for ticker in best_performers['Ticker']:
    try:
        df = pd.read_csv(f'stock_data/{ticker}.csv', index_col = 0)
        moving_avg = [150, 200]
        for ma in moving_avg:
            df['SMA_'+str(ma)] = round(df['Adj Close'].rolling(window = int(ma)).mean(), 2)
        latest_price = df['Adj Close'][-1]
        pe_ratio = float(si.get_quote_table(ticker)['PE Ratio (TTM)'])
        peg_ratio = float(si.get_stats_valuation(ticker)[1][4])
        moving_average_150 = df['SMA_150'][-1]
        moving_average_200 = df['SMA_200'][-1]
        low_52week = round(min(df['Low'][-(52*5):]), 2) # 5 days of trading times 52 week in a year
        high_52week = round(max(df['High'][-(52*5):]), 2)
        score = round(best_performers[best_performers['Ticker'] == ticker]['Score'].tolist()[0])
            
        # Conditions
        condition_1 = latest_price > moving_average_150 > moving_average_200
        condition_2 = latest_price >= (1.3 * low_52week)
        condition_3 = latest_price >= (0.75 * high_52week)
        condition_4 = pe_ratio < 40
        condition_5 = peg_ratio < 2
            
        # Conditional
        if condition_4 and condition_5:            #condition_1 and condition_2 and condition_3 and 
            final_df = final_df.append({'Tickers': ticker,
                                        'Latest_Price': latest_price,
                                        'Score': score,
                                        'PE_Ratio': pe_ratio,
                                        'PEG_Ratio': peg_ratio,
                                        'SMA_150': moving_average_150,
                                        'SMA_200': moving_average_200,
                                        '52_week_low': low_52week,
                                        '52_week_high': high_52week}, 
                                         ignore_index = True)
    
    except Exception as e:
        print(f"{e} for {ticker}")
              

list index out of range for ABMD
list index out of range for ADBE
list index out of range for ADP
list index out of range for ANTM
list index out of range for AON
list index out of range for AVGO
list index out of range for AWK
list index out of range for AZO
list index out of range for BKNG
list index out of range for BR
list index out of range for CDNS
list index out of range for CNC
list index out of range for COST
list index out of range for CPRT
list index out of range for DG
list index out of range for DLTR
list index out of range for DXCM
list index out of range for EQIX
list index out of range for ETN
list index out of range for EXR
list index out of range for FTNT
list index out of range for GOOG
list index out of range for HD
list index out of range for HSY
list index out of range for ILMN
list index out of range for INCY
list index out of range for INTU
list index out of range for IT
list index out of range for KR
list index out of range for LLY
list index out of range for L

In [20]:
final_df.sort_values(by = 'Score', ascending = False)
pd.set_option('display.max_columns', 10)
print(final_df)
final_df.to_csv('portfolio/portfolio2-PE-PEG.csv')

  Tickers Latest_Price Score PE_Ratio PEG_Ratio SMA_150 SMA_200 52_week_low  \
0     AME   131.820007    72     31.9      1.68  134.68   134.9       121.8   

  52_week_high  
0       148.07  


# Rough Work

In [21]:
si.get_quote_table(ticker)

{'1y Target Est': 639.09,
 '52 Week Range': '375.63 - 615.00',
 'Ask': '404.94 x 800',
 'Avg. Volume': 474357.0,
 'Beta (5Y Monthly)': 1.53,
 'Bid': '393.35 x 800',
 "Day's Range": '390.27 - 397.35',
 'EPS (TTM)': 15.68,
 'Earnings Date': 'May 02, 2022 - May 06, 2022',
 'Ex-Dividend Date': nan,
 'Forward Dividend & Yield': 'N/A (N/A)',
 'Market Cap': '21.071B',
 'Open': 391.0,
 'PE Ratio (TTM)': 25.14,
 'Previous Close': 393.68,
 'Quote Price': 394.2799987792969,
 'Volume': 262034.0}

In [22]:
si.get_stats_valuation(ticker)

,0,1
0,Market Cap (intraday),NaN
1,Enterprise Value,NaN
2,Trailing P/E,NaN
3,Forward P/E,NaN
4,PEG Ratio (5 yr expected),NaN
5,Price/Sales (ttm),NaN
6,Price/Book (mrq),NaN
7,Enterprise Value/Revenue,NaN
8,Enterprise Value/EBITDA,NaN
